In [30]:
import pandas as pd
import numpy as np
import datetime 
import json

In [31]:
def is_date(strdate):
    '''判断是否是一个有效的日期字符串'''
    try:
        if ":" in strdate:
            datetime.datetime.strptime(strdate, "%Y-%m-%d %H:%M:%S")
        else:
            datetime.datetime.strptime(strdate, "%Y-%m-%d")
        return True
    except:
        return False

In [32]:
c_t = "http://www.w3.org/2000/01/rdf-schema#Class"
p_t = "http://www.w3.org/1999/02/22-rdf-syntax-ns#Property"
c_xml = "http://www.openkg.cn/COVID-19/character/class/"
p_xml = "http://www.openkg.cn/COVID-19/character/property/"
r_xml = "http://www.openkg.cn/COVID-19/character/resource/"

path = r"data/英雄.json"

In [33]:
'''创建Map'''
class_map = {}
property_map = {}

In [34]:
with open(path, encoding='utf-8') as f:
    f = json.load(f)['@graph']
    for G in f:
        if G['@type'] == c_t:
            if G['@id'].find(c_xml) != -1:
                class_map[G['@id'].split(c_xml)[-1]] = G['label']['@value']
        elif G['@type'] == p_t:
            if G['@id'].find(p_xml) != -1:
                property_map[G['@id'].split(p_xml)[-1]] = G['label']['@value']
            else:
                property_map[G['@id'].replace(':','')] = G['label']['@value']
        else:
            pass

In [35]:
entity_id = []
entity_value = []
entity_label = []
rel_from = []
rel_to = []
rel_type = []
dateC = 1
stringC = 1

entity = pd.DataFrame()
rel = pd.DataFrame()

In [36]:
with open(path, encoding='utf-8') as f:
    f = json.load(f)['@graph']
    for G in f:
        if G['@type'].find(c_xml)!= -1:
            r_id = G['@id'].split(r_xml)[-1] # main resource的 RID
            entity_id.append(r_id)
            entity_value.append(G['label']['@value'])
            entity_label.append(class_map[G['@type'].split(c_xml)[-1]])
            for k in G.keys():
                if k.startswith('P'):
                    # 一个PID可能有多个对应的Resource 或者是单独的String/Date
                    if type(G[k]) == str:
                        rel_from.append(r_id)
                        rel_type.append(property_map[k]) # 创建main resource和property
                        if is_date(G[k]) == True: # 若是date字符串则 需要创建entity 并设置RID
                            dr_id = 'character_dR'+ str(dateC) # Character_Date dR+ID
                            dateC += 1
                            entity_id.append(dr_id)
                            entity_value.append(G[k])
                            entity_label.append('Character_Date')
                            rel_to.append(dr_id) # 记得加上rel_to 连接main resource
                            
                        elif G[k].find(r_xml) == -1:
                        # 若是String的值 同样需要创建entity
                            sr_id = 'character_sR'+ str(stringC) # Character_String sR+ID
                            stringC += 1
                            entity_id.append(sr_id)
                            entity_value.append(G[k])
                            entity_label.append('Character_String')
                            rel_to.append(sr_id) # 记得加上rel_to 连接main resource
                        else:
                            rel_to.append(G[k].split(r_xml)[-1])
                    else: # 若不是str那么就是list, 即url_list
                        for url in G[k]:
                            rel_from.append(r_id)
                            rel_type.append(property_map[k]) # 创建main resource和property
                            rel_to.append(url.split(r_xml)[-1])

In [37]:
print(len(entity_id))
print(len(entity_value))
print(len(entity_label))
print(len(rel_from))
print(len(rel_to))
print(len(rel_type))

1036
1036
1036
1449
1449
1449


In [38]:
entity['Character:ID(Character-ID)'] = entity_id
entity['value'] = entity_value
entity[':LABEL'] = entity_label
rel[':START_ID(Character-ID)'] = rel_from
rel[':END_ID(Character-ID)'] = rel_to
rel[':TYPE'] = rel_type

In [39]:
entity.to_csv(r'output/character_entity.csv', encoding='utf-8', index=False)
rel.to_csv(r'output/character_rel.csv', encoding='utf-8',  index=False)

In [ ]:
neo4j-admin import --mode=csv --multiline-fields=true 
--nodes import\character_entity.csv --nodes import\event_entity.csv  
--nodes import\concept_entity.csv --nodes import\goods_entity.csv  
--nodes import\health_entity.csv --nodes import\medical_entity.csv  
--nodes import\prevention_entity.csv --nodes import\wiki_entity.csv  

--relationships import\character_rel.csv --relationships import\event_rel.csv
--relationships import\concept_rel.csv --relationships import\goods_rel.csv
--relationships import\health_rel.csv --relationships import\medical_rel.csv
--relationships import\prevention_rel.csv --relationships import\wiki_rel.csv

In [ ]:
neo4j-admin import --mode=csv --multiline-fields=true --nodes import\character_entity.csv --nodes import\event_entity.csv  --nodes import\concept_entity.csv --nodes import\goods_entity.csv --nodes import\health_entity.csv --nodes import\medical_entity.csv  --nodes import\prevention_entity.csv --nodes import\wiki_entity.csv  --relationships import\character_rel.csv --relationships import\event_rel.csv --relationships import\concept_rel.csv --relationships import\goods_rel.csv --relationships import\health_rel.csv --relationships import\medical_rel.csv --relationships import\prevention_rel.csv --relationships import\wiki_rel.csv